# 🛍️ Segmentación de Clientes de E-Commerce

Este proyecto tiene como objetivo identificar distintos grupos de clientes con base en su edad, ingresos y comportamiento de compra, utilizando **clustering (K-Means)** y análisis exploratorio de datos (**EDA**) en Python.

Se busca responder preguntas como:
- ¿Qué tipo de clientes gastan más?
- ¿Qué grupos tienen mayor tasa de abandono?
- ¿Qué estrategias podrían ayudar a retener a los clientes más valiosos?



In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/data/ecommerce_customers_dirty.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
def detect_outliers_iqr(df, columna):

    q1 = df[columna].quantile(0.25)
    q3 = df[columna].quantile(0.75)
    iqr = q3 - q1

    limite_inferior = q1 - 1.5*iqr
    limite_superior = q3 + 1.5*iqr

    df[f"{columna}_es_outlier"] = (df[columna] < limite_inferior) | (df[columna] > limite_superior)

    print(f"🔍 Outliers detectados en '{columna}': {df[f'{columna}_es_outlier'].sum()}")
    return df

In [ ]:
# Detectar outliers en Age

df = detect_outliers_iqr(df, "Age")

In [ ]:
df[df["Age_es_outlier"] == True]

In [ ]:
# Imputar con mediana los valores negativos y nulos

mediana_edad = df['Age'].median()
df.loc[df['Age'] < 0, 'Age'] = mediana_edad

df['Age'] = df['Age'].fillna(mediana_edad)

In [ ]:
# Imputar nulos con moda para el genero

moda_genero = df['Gender'].mode()[0]
df['Gender'] = df['Gender'].fillna(moda_genero)

In [ ]:
# Detectar outliers en Spending_Score

df = detect_outliers_iqr(df, "Spending_Score")
df[df["Spending_Score_es_outlier"] == True]

In [ ]:
# Imputar con mediana los valores negativos

mediana_ss = df['Age'].median()
df.loc[df['Spending_Score'] < 0, 'Spending_Score'] = mediana_ss

In [ ]:
# Detectar outliers en Annual_Income_USD

df = detect_outliers_iqr(df, "Annual_Income_USD")
df[df["Annual_Income_USD_es_outlier"] == True]

In [ ]:
# Imputar con mediana los valores negativos y nulos

mediana_aiusd = df['Annual_Income_USD'].median()
df.loc[df['Annual_Income_USD'] < 0, 'Annual_Income_USD'] = mediana_aiusd

df['Annual_Income_USD'] = df['Annual_Income_USD'].fillna(mediana_aiusd)

In [ ]:
# Imputar con moda los nulos en la columna Churned

moda_churned = df['Churned'].mode()[0]
df['Churned'] = df['Churned'].fillna(moda_churned)

In [ ]:
df["Country"].unique()
df["Country"].value_counts()

In [ ]:
# Eliminar registros de edad atipicos y filtrar
df = df[df["Age"] >= 18]

In [ ]:
# Eliminar columnas que ya no se necesitan

df.drop(columns = ["Spending_Score_es_outlier", "Age_es_outlier", "Annual_Income_USD_es_outlier"], inplace = True)

In [ ]:
# Gurdar CSV limpio

df.to_csv("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/data/clientes_ecommerce_limpio.csv", index = False)

In [ ]:
# Histplot y boxplot de Age

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (12,4))

plt.subplot(1, 2, 1)
sns.histplot(df["Age"], bins = 30, kde = True, color = "skyblue" )
plt.title("Histograma de Edad")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")

plt.subplot(1, 2, 2)
sns.boxplot(x = df["Age"], color = 'salmon')
plt.title("Boxplot de Edad")

plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/histplot_boxplot_age")
plt.show()

In [ ]:
# Histplot y boxplot de Annual_Income_USD

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (12,4))

plt.subplot(1, 2, 1)
sns.histplot(df["Annual_Income_USD"], bins = 30, kde = True, color = "skyblue" )
plt.title("Distribucion de Ingresos Anuales en USD")
plt.xlabel("Ingresos Anuales en USD")
plt.ylabel("Frecuencia")

plt.subplot(1, 2, 2)
sns.boxplot(x = df["Annual_Income_USD"], color = 'salmon')
plt.title("Histograma de Ingresos Anuales en USD")

plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/histplot_boxplot_Annual_Income_USD")
plt.show()

In [ ]:
# Histplot y boxplot de Spending_Score

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (12,4))

plt.subplot(1, 2, 1)
sns.histplot(df["Spending_Score"], bins = 30, kde = True, color = "skyblue" )
plt.title("Distribucion de Puntuación de gasto")
plt.xlabel("Puntuación de gasto")
plt.ylabel("Frecuencia")

plt.subplot(1, 2, 2)
sns.boxplot(x = df["Spending_Score"], color = 'salmon')
plt.title("Histograma de Puntuación de gasto")

plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/histplot_boxplot_Spending_Score")
plt.show()

In [ ]:
# Distribucion de Edad, Ingreso, Score segun Churned

fig, axs = plt.subplots(3, 1, figsize = (10,12))

columnas = ["Age", "Annual_Income_USD", "Spending_Score"]
colores = ["#f94144", "#577590"]

for i, col in enumerate(columnas):
    sns.histplot(

        data = df,
        x = col,
        hue = 'Churned',
        kde = True,
        bins = 30,
        alpha = 0.6,
        palette = colores,
        ax = axs[i] )

    axs[i].set_title(f"{col} por Churned")
    axs[i].set_xlabel(col)
    axs[i].set_ylabel("Frecuencia")
    
plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/histplot_age_ss_aiusd_churned")
plt.tight_layout()
plt.show()

In [ ]:
# Determinar las medias

df.groupby("Churned")[["Age", "Annual_Income_USD", "Spending_Score"]].agg(["mean", "count", "std"])

In [ ]:
# Boxplot de Edad, Ingreso, Score segun la media

fig, axs = plt.subplots(3, 1, figsize = (10,20))

columnas = ["Age", "Annual_Income_USD", "Spending_Score"]
colores = ["#f94144", "#577590"]

for i, col in enumerate(columnas):
    sns.boxplot(data = df, x = "Churned", y = col, hue="Churned", palette = colores, legend=False, ax=axs[i])
    sns.stripplot(data=df, x="Churned", y=col, color='gray', alpha=0.3, jitter=0.2, ax=axs[i])

    media_0 = df[df["Churned"] == 0.0][col].mean()
    media_1 = df[df["Churned"] == 1.0][col].mean()

    # Anotar media sobre cada grupo
    axs[i].text(x=0, y=media_0 + 2, s=f"Media: {media_0:.2f}", ha='center', color='black')
    axs[i].text(x=1, y=media_1 + 2, s=f"Media: {media_1:.2f}", ha='center', color='black')

    axs[i].set_title(f"{col} por grupo de Churned (0 = se quedó, 1 = se fue)")
    axs[i].set_xlabel("Churned")
    axs[i].set_ylabel(col)

    
plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/boxplot_age_ss_aiusd_mean")
plt.show()

In [ ]:
# T -Test (Diferencia entre dos grupos es real o suerte)

from scipy.stats import ttest_ind

grupo_0 = df[df["Churned"] == 0]
grupo_1 = df[df["Churned"] == 1]

resultados = {

    "Age": ttest_ind(grupo_0["Age"], grupo_1["Age"], equal_var=False),
    "Annual_Income_USD": ttest_ind(grupo_0["Annual_Income_USD"], grupo_1["Annual_Income_USD"], equal_var=False),
    "Spending_Score": ttest_ind(grupo_0["Spending_Score"], grupo_1["Spending_Score"],equal_var=False)

}

df_resultados = pd.DataFrame({
    "Variable": resultados.keys(),
    "t-statistic": [r.statistic for r in resultados.values()],
    "p-value": [r.pvalue for r in resultados.values()]
})

print(df_resultados)

In [ ]:
# Heatmap de correlacion

df_num = df[["Age", "Annual_Income_USD", "Spending_Score", "Churned"]]
correlaciones = df_num.corr()

plt.figure(figsize = (8,6))
sns.heatmap(
    correlaciones,
    annot = True, #Mostrar los valores dentro del mapa
    cmap = "coolwarm", #Colores de azul a rojo
    fmt = ".2f", #Formato a dos decimales
    linewidths = 0.5 #Bordes entre casillas
)

plt.title("Mapa de Correlación entre Variables Numéricas")
plt.tight_layout()

plt.savefig("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/img/heatmap_variables_numericas")
plt.show()


    
    

In [ ]:
# Clustering

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
os.environ["OMP_NUM_THREADS"] = "2"

#Seleccionamos solo las variables numéricas que nos interesan para agrupar clientes
df_filtrado = df[df["Age"] >= 18].copy()
X = df_filtrado[["Age", "Annual_Income_USD", "Spending_Score"]]

# Esto convierte todas las columnas a una misma escala (media 0, desviación 1). Así evitamos que Annual_Income_USD pese más que las otras
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# modelo de K-Means
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)

# fit_predict() entrena el modelo y asigna cada cliente a un cluster. Guardamos el número de grupo (0, 1, 2) en una nueva columna "Cluster"
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Contamos cuántos clientes hay en cada grupo
conteo_clusters = df_filtrado["Cluster"].value_counts().sort_index()

In [ ]:
# Gráfico: ingresos vs. score de gasto, coloreado por cluster
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df,
    x="Annual_Income_USD",
    y="Spending_Score",
    hue="Cluster",
    palette="Set2",
    s=80,
    alpha=0.8
)

plt.title("Segmentación de Clientes por Ingreso vs Score de Gasto")
plt.xlabel("Ingreso Anual (USD)")
plt.ylabel("Spending Score")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

In [ ]:
#Cruzar clusters con churn
df.groupby("Cluster")["Churned"].mean()

In [ ]:
#Perfilado por cluster

df.groupby("Cluster")[["Age", "Annual_Income_USD", "Spending_Score"]].mean()

In [ ]:
df_filtrado.to_csv("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/data/clientes_segmentados.csv", index=False)

In [ ]:
# Perfil Promedio Por Cluster

df[df["Cluster"] == 2].groupby("Churned")[["Age", "Annual_Income_USD"]].mean()

In [ ]:
#cruzamos Cluster con Churned para calcular la tasa de abandono por grupo

df.groupby("Cluster")["Churned"].mean()


In [ ]:
# Agrupar por cluster y calcular medias
perfil_clusters = df.groupby("Cluster")[["Age", "Annual_Income_USD", "Spending_Score", "Churned"]].mean().reset_index()

# Renombrar columnas para que sea más entendible
perfil_clusters.columns = [
    "Cluster",
    "Edad_promedio",
    "Ingreso_promedio",
    "Score_promedio",
    "Tasa_de_Churn"
]

# Mostrar resultados
print(perfil_clusters)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Normalizar los valores de 0 a 1 por columna (para que todo entre en el radar)
perfil_normalizado = perfil_clusters.copy()
variables = ["Edad_promedio", "Ingreso_promedio", "Score_promedio", "Tasa_de_Churn"]

for col in variables:
    min_val = perfil_clusters[col].min()
    max_val = perfil_clusters[col].max()
    perfil_normalizado[col] = (perfil_clusters[col] - min_val) / (max_val - min_val)

# Preparar datos para radar
labels = variables
num_vars = len(labels)

# Ángulos para cada eje del radar
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # cerrar el círculo

# Plot
plt.figure(figsize=(8, 6))
for i, row in perfil_normalizado.iterrows():
    valores = row[variables].tolist()
    valores += valores[:1]  # cerrar el gráfico
    plt.polar(angles, valores, label=f"Cluster {int(row['Cluster'])}", linewidth=2)
    plt.fill(angles, valores, alpha=0.1)

plt.xticks(angles[:-1], labels)
plt.title("Perfil Promedio Normalizado por Cluster (Radar Chart)")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(data=df, x="Annual_Income_USD", y="Spending_Score", hue="Cluster")

In [ ]:
# Calcular tasa de churn por cluster
churn_por_cluster = df.groupby("Cluster")["Churned"].mean().reset_index()
churn_por_cluster.columns = ["Cluster", "Tasa_de_Churn"]

# Gráfico de barras horizontales
plt.figure(figsize=(8, 5))
sns.barplot(
    data=churn_por_cluster,
    y="Cluster",
    x="Tasa_de_Churn",
    palette="coolwarm",
    hue="Tasa_de_Churn"
)
plt.title("Tasa de Churn por Cluster")
plt.xlabel("Tasa de Churn")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px

# Mapa de clientes por pais
# Agrupar por país y contar clientes
clientes_por_pais = df["Country"].value_counts().reset_index()
clientes_por_pais.columns = ["Country", "Clientes"]

# Crear mapa
fig = px.choropleth(
    clientes_por_pais,
    locations="Country",
    locationmode="country names",  # O usa "ISO-3" si tienes códigos
    color="Clientes",
    color_continuous_scale="Blues",
    title="Distribución de Clientes por País"
)
fig.show()

In [ ]:
perfil_clusters = df.groupby("Cluster")[["Age", "Annual_Income_USD", "Spending_Score", "Churned"]].mean().reset_index()
perfil_clusters.columns = ["Cluster", "Edad_promedio", "Ingreso_promedio", "Score_promedio", "Tasa_de_Churn"]

perfil_clusters.to_csv("C:/Users/migue/OneDrive/Escritorio/Analisis Datos/Semana_5/data/perfil_clusters.csv", index=False)

## 🧠 Conclusiones

- El **Cluster 0** representa a clientes con ingresos bajos pero alto gasto. Son los más rentables y con menor tasa de abandono.
- El **Cluster 2** tiene ingresos más altos, pero gasta menos y presenta la tasa de abandono más alta. Podría beneficiarse de estrategias de fidelización.
- El **Cluster 1** es neutral en todos los sentidos, con valores intermedios.

Estos insights permiten a las empresas **personalizar sus campañas** y **optimizar la retención de clientes** a partir de datos reales.